# Database test

## Simulation description file

In [26]:
DOI="10.5281/zenodo.3557459"
def_file  = "/home/osollila/NMRlipids/MATCH/scripts/orderParm_defs/order_parameter_definitions_MODEL_CHARMM36_DOPE.def"

user_information = """
DOPE test
#NMRLIPIDS BEGIN

@SIM
@SOFTWARE=gromacs
@FF=CHARMM36
@FF_SOURCE=CHARMM-GUI
@FF_DATE=??
@TRJ=dope.trr
@TPR=dope.tpr

#NMRLIPIDS END

"""

## Working directory

In [2]:
# Working directory
dir_wrk  = "/media/osollila/DATADRIVE1/temp3"

## General Imports

In [3]:
# Working with files and directories
import os

#For quering webs
import urllib.request
from urllib.error import URLError,HTTPError

# From time monitoring
from tqdm import tqdm

# Python program to find SHA256 hash string of a file
import hashlib

# For dealing with excel and cvs 
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

#To make real independent copies of lists
from copy import deepcopy

## Directories

In [4]:
dir_wrk = os.path.normpath(dir_wrk)
print(dir_wrk)
def_file = os.path.normpath(def_file)
print(def_file)

/media/osollila/DATADRIVE1/temp3
/home/osollila/NMRlipids/MATCH/scripts/orderParm_defs/order_parameter_definitions_MODEL_CHARMM36_POPE.def


## Check that the DOI link is valid

In [5]:
DOI_url = 'https://doi.org/' + DOI
print(DOI_url)

try:
    response = urllib.request.urlopen(DOI_url)
    print("Status of the DOI link: {0}".format(response.msg))
except HTTPError as e:
    print(DOI_url)
    print('The server couldn\'t fulfill the request.')
    print('Error code: ', e.code)
    user_information = ""
    print('The code will not proceed, please fix DOI')
except URLError as e:
    print(DOI_url)
    print('We failed to reach a server.')
    print('Reason: ', e.reason)
    user_information = ""
    print('The code will not proceed, please fix DOI')
else:
    pass

https://doi.org/10.5281/zenodo.3557459
Status of the DOI link: OK


## Read input description

In [6]:
bNMRLIPIDS = False #Check if the link contains NMRLIPIDS metadata
nsims =0 # Counter number of simulations in a submission
sims = [] #Array with the dictionary containing the information of a simulation

for line in user_information.split("\n"):
    if line.strip() == "":
        continue
    if "#NMRLIPIDS BEGIN" in line:
        bNMRLIPIDS = True
        continue
    if "#NMRLIPIDS END" in line:
        bNMRLIPIDS = False
        continue
    if "@SIM" in line:
        #sims.append({"ID" : nsims, "STATUS" : 0})
        sims.append({"ID" : nsims})
        nsims += 1
        continue
    if not bNMRLIPIDS:
        continue
    if line.strip()[0] == "@":
        key, value = line.split("=")
        sims[-1][key.strip('@')] = value
print(nsims)
print(sims)      
        

1
[{'ID': 0, 'SOFTWARE': 'gromacs', 'FF': 'CHARMM36', 'FF_SOURCE': 'CHARMM-GUI', 'FF_DATE': '??', 'TRJ': 'dope.trr', 'TPR': 'dope.tpr'}]


### Dictionares

In [7]:
# Gromacs
gromacs_dict = {
               'INI' : {"REQUIRED": False,
                        "TYPE" : "files",
                        "EXTENSION" : ("gro", "pdb",),
                       }, # Could be not needed in the future (tpr)
               'MDP' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("mdp",),
                       }, # Could be not needed in the future (tpr)
               'TRJ' : {"REQUIRED": True,
                        "TYPE" : "files",
                        "EXTENSION" : ("xtc","trr",),
                       },
               'TPR' : {"REQUIRED": True,
                        "TYPE" : "file",
                        "EXTENSION" : ("tpr",),
                       },
               'CPT' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("cpt",),
                       },
               'TOP' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("top",),
                       },
               'ITP' : {"REQUIRED": False,
                        "TYPE" : "files",
                        "EXTENSION" : ("itp",),
                       },
               'FF'  : {"REQUIRED": False,
                        "TYPE" : "string",
                       },
               'FF_SOURCE' : {"REQUIRED": False,
                              "TYPE" : "string",
                              },
               'FF_DATE' : {"REQUIRED": False,
                            "TYPE" : "date",
                           },
               }

# Amber
amber_dict = {}

# NAMD
namd_dict = {}

# CHARMM
charmm_dict = {}

# OPENMM
openmm_dict = {}

# SOFTWARE
software_dict = {
                "GROMACS" : gromacs_dict, 
                "AMBER"   : amber_dict,
                "NAMD"    : namd_dict,
                "CHARMM"  : charmm_dict,
                "OPENMM"  : openmm_dict,
                }

print(software_dict.keys())

dict_keys(['GROMACS', 'AMBER', 'NAMD', 'CHARMM', 'OPENMM'])


### Check software used by the simulation

In [8]:
sims_valid_software = []
for sim in sims:
    if sim['SOFTWARE'].upper() in software_dict.keys():
        msg_info = "Simulation {0} uses supported software {1} and will be further processed"
        print (msg_info.format(sim['ID'], sim['SOFTWARE'].upper()))
        sims_valid_software.append(sim.copy())
    else:
        msg_err="Simulation {0} performed in an UNSUPPORTED software {1} and will NOT be further processed"
        print(msg_err.format(sim["ID"], sim["SOFTWARE"].upper()))
#print(sims_valid_software) 

Simulation 0 uses supported software GROMACS and will be further processed


### Check that all entry keys provided for each simulation are valid:

In [9]:
sims_valid_entries = []
for sim in sims_valid_software:
    #print("ID {0}".format(sim["ID"]))
    wrong_key_entries = 0
    software_dict_name = "{0}_dict".format(sim['SOFTWARE'].lower())
    for key_sim, value_sim in sim.items():
        #print(key_sim, value_sim)
        if key_sim.upper() in ("ID", "SOFTWARE"):
            #print("NOT REQUIRED")
            continue
        if key_sim.upper() not in software_dict[sim['SOFTWARE'].upper()].keys():
            print ("{0} NOT in {1}".format(key_sim, software_dict_name))
            wrong_key_entries += 1
    if wrong_key_entries:
        print("Simulation {0} has {1} unknown entry/ies and won't be longer considered, please correct.\n".format(sim['ID'],wrong_key_entries))
    else:
        msg_info = "All entries in simulation {0} are understood and will be further processed\n"
        print (msg_info.format(sim['ID']))
        sims_valid_entries.append(sim.copy())
#print(sims_valid_entries)

All entries in simulation 0 are understood and will be further processed



### Process entries with files information to contain file names in arrays

In [10]:
sims_files_to_array = deepcopy(sims_valid_entries)

for sim in sims_files_to_array:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        try:
            entry_type = software_sim[key_sim]['TYPE']
            if "file" in entry_type:
                if isinstance(value_sim, list): continue  
                files_list = []
                print("{0} added to list".format(value_sim))
                # Place filenames into arrays
                for file_provided in value_sim.split(";"):
                    files_list.append([file_provided.strip()])
                sim[key_sim] = files_list
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
#print(sims_files_to_array)
#print(sims_valid_entries)

ID 0
dope.trr added to list
dope.tpr added to list


### Check for multiple files in entries that can only contain one

In [11]:
sims_valid_file_entries = []
for sim in sims_files_to_array:
    print("ID {0}".format(sim["ID"]), flush=True)
    files_issues = 0
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        try:
            entry_type = software_sim[key_sim]['TYPE']
            if entry_type == "file"  and len(value_sim) > 1:
                print("Multiple values found in {0} and only one allowed (Please correct):\n {1}".format(key_sim,value_sim))
                files_issues += 1
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    if files_issues:
        print("Sim {0} will be no longer processed".format(sim["ID"]))
    else:
        sims_valid_file_entries.append(sim.copy())
#print(sims_valid_file_entries)

ID 0


### Check if the submitted simulation has rssion has all required files and information

In [12]:
sims_required_entries = []
for sim in sims_valid_file_entries:
    print("ID {0}".format(sim["ID"]))
    missing_required_keys = 0
    for key, value in software_dict[sim['SOFTWARE'].upper()].items():
        if value["REQUIRED"]:
            try:
                sim[key]
            except:
                print("Entry not found: {0} {1}".format(key, value))
                missing_required_keys += 1
    if missing_required_keys:
        print("{0} missing required entry/ies, please correct.".format(missing_required_keys))
        print("Entry with ID={0} will not be further processed.\n".format(sim["ID"]))
    else:
        print("All required entries present.\n")
        sims_required_entries.append(sim.copy())

ID 0
All required entries present.



### Check status links

In [14]:
# Download link
def download_link(doi, file):
    if "zenodo" in doi.lower():
        zenodo_entry_number = doi.split(".")[2]
        return 'https://zenodo.org/record/' + zenodo_entry_number + '/files/' + file
    else:
        print ("DOI provided: {0}".format(doi))
        print ("Repository not validated. Please upload the data for example to zenodo.org")
        return ""

In [15]:
#print(sims_required_entries)
sims_working_links = []         
for sim in sims_required_entries:
    print("ID {0}".format(sim["ID"]))
    wrong_links = 0
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                for file_provided in value_sim:
                    #print("File={0}".format(file_provided[0]))
                    file_url = download_link(DOI, file_provided[0])
                    if file_url == "":
                        wrong_links += 1
                        continue
                    try:
                        response = urllib.request.urlopen(file_url)
                        #print("Status of the DOI link: {0}".format(response.msg))
                    except HTTPError as e:
                        print("\nkey={0} => file={1}".format(key_sim, file_provided[0]))
                        print(file_url)
                        print('The server couldn\'t fulfill the request.')
                        print('Error code: ', e.code)
                        wrong_links += 1
                    except URLError as e:
                        print(key_sim, file_provided[0])
                        print(file_url)
                        print('We failed to reach a server.')
                        print('Reason: ', e.reason)
                        wrong_links += 1
                    else:
                        pass
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    if wrong_links:
        print("{0} link/s failed, please correct.".format(wrong_links))
        print("Sim={0} will not be further processed.\n".format(sim["ID"]))
    else:
        print("All links work.\n")
        sims_working_links.append(sim.copy())
#print(sims_working_links)

ID 0
All links work.



## Download files from links

In [15]:
# Create temporary directory where to download files and analyze them
dir_tmp = os.path.join(dir_wrk, "tmp/")
if (not os.path.isdir(dir_tmp)): os.mkdir(dir_tmp)

for sim in sims_working_links:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    dir_sim = os.path.join(dir_tmp, str(sim["ID"]))
    if (not os.path.isdir(dir_sim)): os.mkdir(dir_sim)
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                for file_provided in tqdm(value_sim, desc = key_sim):
                    file_url = download_link(DOI, file_provided[0])
                    file_name = os.path.join(dir_sim, file_provided[0])
                    if (not os.path.isfile(file_name)):
                        response = urllib.request.urlretrieve(file_url, file_name)
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
            

ID 0


TPR: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s]


## Calculate hash downloaded files

In [16]:
dir_tmp = os.path.join(dir_wrk, "tmp/")
sims_hashes = deepcopy(sims_working_links)

for sim in sims_hashes:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    dir_sim = os.path.join(dir_tmp, str(sim["ID"]))
    
    #list_containing the sha1 sums for all required files
    sha1_list_requied = []
    
    # Make empty dataframe with the desired columns
    df_files = pd.DataFrame(columns=['NAME','TYPE','REQUIRED','HASH'])
    
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                files_list = []
                for file_provided in value_sim:
                    file_name = os.path.join(dir_sim, file_provided[0])
                    sha1_hash = hashlib.sha1()
                    with open(file_name,"rb") as f:
                        # Read and update hash string value in blocks of 4K
                        for byte_block in iter(lambda: f.read(4096),b""):
                            sha1_hash.update(byte_block)
                        #print(file_provided, sha256_hash.hexdigest())
                        df_files = df_files.append({
                            "NAME":file_provided[0],
                            "TYPE":key_sim,
                            "REQUIRED": software_dict[sim['SOFTWARE'].upper()][key_sim]['REQUIRED'],
                            "HASH":sha1_hash.hexdigest(),
                        }, ignore_index=True)
                    files_list.append([file_provided[0], sha1_hash.hexdigest()])
                #Find the keys of the required files to calculate the master_hash 
                if software_dict[sim['SOFTWARE'].upper()][key_sim]['REQUIRED'] == True:
                    sha1_list_requied.append(sha1_hash.hexdigest())
                sim[key_sim] = files_list #Problematic
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    print(df_files)
    print("\n{0}\n".format(sha1_list_requied))      
    # Calculate the hash of a file contaning the hashes of each of the required files
    # This should be always invariant as it will be used unique identifier for a simualtion
    # Note order the hashes of the required files before calculating the hash (That means that the required files cannot change)
print(sims_hashes)

ID 0
       NAME TYPE REQUIRED                                      HASH
0  dope.trr  TRJ     True  e228e0e15becc30a6870ef7f0d6261e7c7eafd4c
1  dope.tpr  TPR     True  5095cc60210597caff7914ea1e9ddd7beaeb6043

['e228e0e15becc30a6870ef7f0d6261e7c7eafd4c', '5095cc60210597caff7914ea1e9ddd7beaeb6043']

[{'ID': 0, 'SOFTWARE': 'gromacs', 'FF': 'CHARMM36', 'FF_SOURCE': 'CHARMM-GUI', 'FF_DATE': '??', 'TRJ': [['dope.trr', 'e228e0e15becc30a6870ef7f0d6261e7c7eafd4c']], 'TPR': [['dope.tpr', '5095cc60210597caff7914ea1e9ddd7beaeb6043']]}]


In [57]:
str(sims_working_links)

"[{'ID': 0, 'SOFTWARE': 'gromacs', 'FF': 'CHARMM36', 'FF_SOURCE': 'CHARMM-GUI', 'FF_DATE': '??', 'TRJ': [['dope.trr']], 'TPR': [['dope.tpr']]}]"

# Save to databank

In [58]:
for sim in sims_working_links:
    ID=sim.get('ID')
    # SAMULI: I replace "/" with ":" because the former cannot be used in trajectory name
    DATAdir='../Data/Simulations/'+str(DOI.replace("/",":"))+'.'+str(ID)
    !mkdir {DATAdir}
    # SAMULI: I am writin now in txt, but using json might be better in the future
    outfileDICT=open(str(DATAdir)+'/README.md','w')
    outfileDICT.write(str(sim))
    outfileDICT.close()

mkdir: cannot create directory ‘../Data/Simulations/10.5281:zenodo.3557459.0’: File exists


# Analysis starts here
## First dowload packages and calculate the correlation functions using gromacs tools

In [27]:
from OrderParameter import *
import warnings
from corrtimes import *
import subprocess
!cp corr_ftios_ind.sh {dir_wrk}
for sim in sims_working_links:
    trj=sim.get('TRJ')
    tpr=sim.get('TPR')
    ID=sim.get('ID')
    
    #tprnowat=str(dir_wrk)+'/tmp/'+str(ID)+'/'+'nowater.tpr'
    #xtcnowatwhole=str(dir_wrk)+'/tmp/'+str(ID)+'/'+'whole.xtc'
    xtcwhole=str(dir_wrk)+'/tmp/'+str(ID)+'/'+'whole.xtc'
    #trj='tmp/'+str(ID)+'/'+str(trj[0][0])
    tpr=str(dir_wrk)+'/tmp/'+str(ID)+'/'+str(tpr[0][0])
    #print(def_file,tpr,trj)

    # SAMULI: line below could be removed by giving output folder for corr_ftios.sh
    #!mv *rotacf*xvg {dir_wrk}/tmp/{ID}/
    # SAMULI: line below could are because find_OP does not work for folders
    #!cp {dir_wrk}/tmp/{ID}/{str(trj[0][0])} ./
    #!cp {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} ./
    #Hanne: shortening the traj here for debugging 
    !echo System | gmx trjconv -f {dir_wrk}/tmp/{ID}/{str(trj[0][0])} -s {tpr} -o {xtcwhole} -pbc mol 
    # SAMULI: I compromized the speed to generality. Not all systems automatically have non-water group
    #!echo non-water | gmx trjconv -f {dir_wrk}/tmp/{ID}/{str(trj[0][0])} -t0 0 -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {xtcnowatwhole} -pbc mol 
    #!echo non-Water | gmx trjconv -f tmp.trr -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {dir_wrk}/tmp/{ID}/whole.xtc -pbc mol 
    #!echo non-Water | gmx convert-tpr -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {tprnowat}
    #!rm tmp.trr 

    
    
    output_corr=!{dir_wrk}/corr_ftios_ind.sh {def_file} {tpr} {xtcwhole} {dir_wrk}/tmp/{ID}
    #tmp=output_corr.split()
    #print (output_corr)
    tmp=output_corr[-1].split()
    
    firstlipid=int(tmp[-2])
    lastlipid=int(tmp[-1])
    numlipid=lastlipid-firstlipid+1    
    
    print (ID, numlipid)
    
    
    OrdParam=find_OP(def_file,tpr,xtcwhole)
    outfile=open(str(dir_wrk)+'/tmp/'+str(ID)+'/'+'times'+str(ID)+'.txt','w')
    outfile_teffs=open(str(dir_wrk)+'/tmp/'+str(ID)+'/times'+str(ID)+'_teffs.txt','w')

    
    for i,op in enumerate(OrdParam.values()):
        resops =op.get_op_res
        (op.avg, op.std, op.stem) =op.get_avg_std_stem_OP
        
        outfile_ind=open(str(dir_wrk)+'/tmp/'+str(ID)+'/'+op.name+'times_perlipid.txt','w')
        convs=[]
        Teffs=[]
        Teffs_area=[]
        R1s=[]
        
        for j in range(firstlipid,lastlipid+1):
            print (op.name, j)
            corrfile=dir_wrk+'/tmp/'+str(ID)+'/'+str(j)+'_rotacf_'+op.name+".xvg"
            if j==1:
                out,times=read_data(corrfile)
                fvals=np.asmatrix(out).T
            else:
                out, times=read_data(corrfile)
                fvals=np.concatenate((fvals,np.asmatrix(out).T),axis=1)
            #Hanne: change to calc only R1 here
            Teff,tau_eff_area, R1, conv =calc_corrtime_noread(out,times, resops[j-1])
            Teffs.append(Teff)
            Teffs_area.append(tau_eff_area)
            R1s.append(R1)
            convs.append(conv)
            
            line=str(op.name)+" "+str(resops[j-1])+" "+str(Teff)+" "+str(tau_eff_area)+" "+str(R1)+" "+str(conv)+ '\n'
            outfile_ind.write(line)
        outfile_ind.close()

        line2=str(op.name)+" "+str(op.avg)+" "+str(op.stem)+" "+str(np.mean(Teffs))+" "+str(np.std(Teffs, ddof=1)/np.sqrt(len(Teffs)-1))+" "+str(np.mean(Teffs_area))+" "+str(np.std(Teffs_area,ddof=1)/np.sqrt(len(Teffs_area)-1))+" "+str(np.mean(R1s))+" "+str(np.std(R1s, ddof=1)/np.sqrt(len(R1s)-1))+" "+str(sum(convs)/float(len(convs)))+'\n'
        outfile.write(line2)

        #analysis of Teff error starts here
        means=np.mean(fvals,axis=1)
        stems=np.std(fvals,axis=1,ddof=1)/np.sqrt(int(numlipid)-1)
        teff,teff_min,teff_max=calc_corrtime_withee(times,means, stems, op.avg, op.stem)
        line3=str(op.name)+" "+str(op.avg)+" "+str(teff)+" "+str(teff_min)+" "+str(teff_max)+'\n'
        outfile_teffs.write(line3)        
        line=str(op.name)+" "+str(op.avg)+" "+str(Teff)+" "+str(tau_eff_area)+" "+str(R1)+'\n'

    !cp {str(dir_wrk)}'/tmp/'{str(ID)}'/times'{str(ID)}'.txt' {DATAdir}
    outfile.close()
    

                     :-) GROMACS - gmx trjconv, 2018.2 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

g3_1 64
g3_1 65
g3_1 66
g3_1 67
g3_1 68
g3_1 69
g3_1 70
g3_1 71
g3_1 72
g3_1 73
g3_1 74
g3_1 75
g3_1 76
g3_1 77
g3_1 78
g3_1 79
g3_1 80
g3_1 81
g3_1 82
g3_1 83
g3_1 84
g3_1 85
g3_1 86
g3_1 87
g3_1 88
g3_1 89
g3_1 90
g3_1 91
g3_1 92
g3_1 93
g3_1 94
g3_1 95
g3_1 96
g3_1 97
g3_1 98
g3_1 99
g3_1 100
g3_1 101
g3_1 102
g3_1 103
g3_1 104
g3_1 105
g3_1 106
g3_1 107
g3_1 108
g3_1 109
g3_1 110
g3_1 111
g3_1 112
g3_1 113
g3_1 114
g3_1 115
g3_1 116
g3_1 117
g3_1 118
g3_1 119
g3_1 120
g3_1 121
g3_1 122
g3_1 123
g3_1 124
g3_1 125
g3_1 126
g3_1 127
g3_1 128
g3_2 1
g3_2 2
g3_2 3
g3_2 4
g3_2 5
g3_2 6
g3_2 7
g3_2 8
g3_2 9
g3_2 10
g3_2 11
g3_2 12
g3_2 13
g3_2 14
g3_2 15
g3_2 16
g3_2 17
g3_2 18
g3_2 19
g3_2 20
g3_2 21
g3_2 22
g3_2 23
g3_2 24
g3_2 25
g3_2 26
g3_2 27
g3_2 28
g3_2 29
g3_2 30
g3_2 31
g3_2 32
g3_2 33
g3_2 34
g3_2 35
g3_2 36
g3_2 37
g3_2 38
g3_2 39
g3_2 40
g3_2 41
g3_2 42
g3_2 43
g3_2 44
g3_2 45
g3_2 46
g3_2 47
g3_2 48
g3_2 49
g3_2 50
g3_2 51
g3_2 52
g3_2 53
g3_2 54
g3_2 55
g3_2 56
g3_2 57
g3_2